## Explore Neighbourhoods in the city of Toronto - Part III

##### Import reqiured python libraries

In [148]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

##### Get table data in the web page in to a dataframe 

In [149]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df = dfs[0]

##### Remove rows with 'Not assigned' Borough

In [150]:
df_new = (df[df['Borough'] != 'Not assigned']).reset_index(drop=True)
#df_new

##### Assign Borough name to Neighbourhoods with 'Not assigned' tag

In [151]:
for index, row in df_new.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        df_new.at[index,'Neighbourhood'] = row['Borough']
#df_new.head(20)

##### Groupby 'Postcode' column and combine rows with Neighbourhoods having assigned the same 'Postcode'

In [152]:
strJoin = lambda x:",".join(x.astype(str))     
df_merged = df_new.groupby(['Postcode','Borough']).agg({"Neighbourhood":strJoin}).reset_index()
df_merged.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [9]:
df_merged.shape

(103, 3)

##### Get Geospacial coordinates of Toronto Neighbourhoods

In [153]:
df_merged.rename(columns = {'Postcode':'Postal Code'}, inplace=True)
#df_merged.head()
df_coords = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
#df_coords.head()

##### Merge Latitude & Longitude data to Toronto Neighbourhoods dataframe

In [154]:
df_new_merged = pd.merge(df_merged, df_coords, on='Postal Code')
#df_new_merged.head()

##### Get Boroughs that include the word 'Toronto'

In [155]:
df_updated = (df_new_merged[df_new_merged['Borough'].str.contains ("Toronto")]).reset_index(drop=True)
df_updated
#df_updated.shape

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


## Segmentation & Clustering of Neighbourhoods in the city of Toronto

##### Download required libraries

In [156]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import json #library to handle json files
from geopy.geocoders import  Nominatim #convert an address in to latitude & longitude values
from pandas.io.json import json_normalize #transform json file in to pandas df
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

### Create a Map of Toronto with Neighbourhoods (only neighbourhoods that include the word 'Toronto') superimposed on top

##### Use geopy library to get latitude & longitude values for Toronto city

In [157]:
address = 'Toronto city, Canada'
geolocator = Nominatim (user_agent = 'Toronto_explorer') #define a user-agent
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print (latitude)
print (longitude)

43.7189883
-79.44157


##### Create Toronto Map

In [158]:
Toronto_map = folium.Map (location = [latitude,longitude], zoom_start = 10)

In [12]:
#Toronto_map

##### Add markers to map

In [159]:
for lat, lng, borough, neigh in zip(df_updated['Latitude'], df_updated['Longitude'],  df_updated['Borough'],  df_updated['Neighbourhood']):
    label = '{},{}'.format (borough, neigh)
    label = folium.Popup (label, parse_html = True)
    folium.CircleMarker ([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color = '#3186cc', fill_opacity=0.7, parse_html=False).add_to(Toronto_map)
    

In [19]:
Toronto_map

### Segment & cluster neighbourhoods in E.g. 'Central Toronto'

##### Get Central Toronto data in to a dataframe

In [160]:
CT_data = df_updated[df_updated['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307
8,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678


##### Get geographical coordinates of Central Toronto

In [161]:
add_CT = 'Central Toronto, Toronto'
geolocator = Nominatim (user_agent='Toronto_explorer')
loc = geolocator.geocode(add_CT)
lat = loc.latitude
long = loc.longitude
print (lat)
print (long)


43.653963
-79.387207


##### Visualize 'Central Toronto' neighbourhoods in a map

In [162]:
CT_map = folium.Map (location = [lat,long], zoom_start = 10)

In [163]:
for lat, lng, label in zip(CT_data['Latitude'], CT_data['Longitude'], CT_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker ([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color = '#3186cc', fill_opacity=0.7, parse_html=False).add_to(CT_map)


In [164]:
CT_map

##### Use Foursquare API to explore the neighbourhoods in 'Central Toronto' & segment/cluster them

In [165]:
#Define Foursquare credentials & version
CLIENT_ID = 'WGX0SR5XECNKWUG4MNHZ3ARPZM11HNONEAHD0DHMHQ1APSLY'
CLIENT_SECRET = 'QEO4BTONLV1KMGBEJEIMG1KPTZFFHACT5WCRUI4VSSDSPRNV'
VERSION = '20180604'


##### Explore 1st neighbourhood in Central Toronto ----> 'Lawrence Park'

In [166]:
CT_data.loc[0,'Neighbourhood']
LP_lat = CT_data.loc[0,'Latitude']
LP_lng = CT_data.loc[0,'Longitude']
LP_name = CT_data.loc[0,'Neighbourhood']

##### Get top 100 venues in 'Lawrence Park' within a radius of 500m

In [167]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,LP_lat,LP_lng,LIMIT,radius)

In [168]:
#Send GET request
results = requests.get(url).json()

In [68]:
#esults

##### Clean json and get data in to a df

In [169]:
#Defining get_category_type function
def get_category_type(row):
    try:
        cat_list = row['categories']
    except:
        cat_list = row['venue.categories']
    if len(cat_list) == 0:
        return None
    else:
        return cat_list[0]['name']

In [170]:
venues_LP = results['response']['groups'][0]['items']
venues_LP_df = json_normalize(venues_LP)
venues_LP_df
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_LP_df = venues_LP_df.loc[:, filtered_columns]

# filter the category for each row
venues_LP_df['venue.categories'] = venues_LP_df.apply(get_category_type, axis=1)

# clean columns
venues_LP_df.columns = [col.split(".")[-1] for col in venues_LP_df.columns]

#nearby_venues.head(10)
venues_LP_df 

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


##### Fuction to repeat the process of getting top 100 venues for all neighbourhoods in Central Toronto

In [171]:
def getNearbyVenues (names,latitudes,longitudes,radius=500):
    venues_list =[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print (name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET,VERSION,lat,lng,LIMIT,radius)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Using 'getNearbyVenues' function and get top 100 venues for all neighbourhoods in 'Central Toronto'

In [172]:
CT_venues = getNearbyVenues (names=CT_data['Neighbourhood'], latitudes=CT_data['Latitude'], longitudes=CT_data['Longitude'],radius=500)
CT_venues.head(20)

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
5,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
6,Davisville North,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel
7,Davisville North,43.712751,-79.390197,Circle K,43.712834,-79.391554,Grocery Store
8,Davisville North,43.712751,-79.390197,Subway,43.708378,-79.390473,Sandwich Place
9,Davisville North,43.712751,-79.390197,Gym,43.713126,-79.393537,Gym


In [173]:
#Total No of venues fetched in the Borough 'Central Toronto' - size of CT_venues df
print ('Total No of top venues in all neighbourhoods of Central Toronto is' , CT_venues.shape[0])

Total No of top venues in all neighbourhoods of Central Toronto is 111


In [174]:
#No of top venues returned for each neighbourhood in Central Toronto
CT_venues.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Davisville,38,38,38,38,38,38
Davisville North,9,9,9,9,9,9
"Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West",14,14,14,14,14,14
"Forest Hill North,Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park,Summerhill East",2,2,2,2,2,2
North Toronto West,16,16,16,16,16,16
Roselawn,2,2,2,2,2,2
"The Annex,North Midtown,Yorkville",23,23,23,23,23,23


In [175]:
#No of unique categories from all returned venues for all neighbourhoods in Central Toronto
print('There are {} unique venue categories'.format (len(CT_venues['Venue Category'].unique())))

There are 61 unique venue categories


##### Analyze each Neighbourhood in detail

In [176]:
#get 'Venue Category' as columns in a new df CT_hotspots
CT_hotspots = pd.get_dummies(CT_venues[['Venue Category']],prefix="",prefix_sep="")
CT_hotspots.head()

,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Dessert Shop,Diner,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [177]:
#Add 'Neighbourhood' column to CT_hotspots df
CT_hotspots['Neighbourhood'] = CT_venues['Neighbourhood']

#move 'Neighbourhood' column to 1st column in CT_hotspots df
fixed_cols = [CT_hotspots.columns[-1]] + list(CT_hotspots.columns[:-1])
CT_hotspots = CT_hotspots[fixed_cols]

CT_hotspots.head()

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Dessert Shop,Diner,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [179]:
#Group rows in CT_hotspots df by 'Neighbourhood' + mean of the frequency of occurance of each venue category
CT_grp = CT_hotspots.groupby('Neighbourhood').mean().reset_index()
CT_grp

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Deli / Bodega,Dessert Shop,Diner,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.052632,0.0000,0.000000,0.052632,0.000000,0.026316,0.026316,0.078947,0.026316,0.026316,0.026316,0.0000,0.026316,0.000000,0.026316,0.0,0.026316,0.026316,0.000000,0.026316,0.0000,0.000000,0.000000,0.0,0.026316,0.052632,0.00,0.000000,0.000000,0.000000,0.0000,0.026316,0.026316,0.105263,0.000000,0.000000,0.0000,0.052632,0.0000,0.078947,0.026316,0.0000,0.0000,0.000000,0.000000,0.052632,0.000000,0.0,0.052632,0.026316,0.00,0.000000,0.000000,0.0000
1,Davisville North,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.111111,0.111111,0.0000,0.000000,0.111111,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.111111,0.000000,0.000000,0.111111,0.000000,0.0000,0.000000,0.0000,0.111111,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.071429,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.071429,0.071429,0.0000,0.000000,0.000000,0.071429,0.000000,0.142857,0.0000,0.000000,0.0000,0.000000,0.000000,0.0000,0.0000,0.071429,0.071429,0.071429,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.071429,0.0000
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.0000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0000
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000

In [180]:
#function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [181]:
#get only top 10 venues for each neighbourhood in Central Toronto in to new df

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
CT_venues_sorted = pd.DataFrame(columns=columns)
CT_venues_sorted['Neighbourhood'] = CT_grp['Neighbourhood']

for ind in np.arange(CT_grp.shape[0]):
    CT_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grp.iloc[ind, :], num_top_venues)

CT_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Restaurant,Thai Restaurant,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Discount Store
1,Davisville North,Hotel,Food & Drink Shop,Sandwich Place,Park,Clothing Store,Playground,Breakfast Spot,Grocery Store,Gym,Gourmet Shop
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Liquor Store,Light Rail Station,Pizza Place,Supermarket,Sports Bar,Fried Chicken Joint
3,"Forest Hill North,Forest Hill West",Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden
5,"Moore Park,Summerhill East",Tennis Court,Playground,Yoga Studio,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden,Fried Chicken Joint
6,North Toronto West,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Salon / Barbershop,Spa
7,Roselawn,Ice Cream Shop,Garden,Hotel,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Fried Chicken Joint,Food & Drink Shop
8,"The Annex,North Midtown,Yorkville",Coffee Shop,Sandwich Place,Café,Pizza Place,American Restaurant,Pharmacy,Liquor Store,Pub,Jewish Restaurant,Cosmetics Shop


### Cluster Neighbourhoods in Central Toronto

##### Run k-means to cluster the neighbourhoods in to 5 clusters

In [182]:
# set number of clusters
kclusters = 3

CT_grp_clustering = CT_grp.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([2, 2, 2, 2, 2, 0, 2, 1, 2], dtype=int32)

In [183]:
# add clustering labels
CT_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CT_merged = CT_data

# merge CT_venues_sorted with CT_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(CT_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

CT_merged # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Hotel,Food & Drink Shop,Sandwich Place,Park,Clothing Store,Playground,Breakfast Spot,Grocery Store,Gym,Gourmet Shop
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Salon / Barbershop,Spa
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Sandwich Place,Dessert Shop,Restaurant,Thai Restaurant,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Discount Store
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,0,Tennis Court,Playground,Yoga Studio,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden,Fried Chicken Joint
5,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,2,Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Liquor Store,Light Rail Station,Pizza Place,Supermarket,Sports Bar,Fried Chicken Joint
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Ice Cream Shop,Garden,Hotel,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Fried Chicken Joint,Food & Drink Shop
7,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,2,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
8,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,2,Coffee Shop,Sandwich Place,Café,Pizza Place,American Restaurant,Pharmacy,Liquor Store,Pub,Jewish Restaurant,Cosmetics Shop


#### Visualize the clusters generated for Central Toronto Neighbourhood Venues

In [184]:
# create map
add_CT = 'Central Toronto, Toronto'
geolocator = Nominatim (user_agent='Toronto_explorer')
loc = geolocator.geocode(add_CT)
lat = loc.latitude
long = loc.longitude
map_clusters_CT = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Neighbourhood'], CT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_CT)
       
map_clusters_CT

#### Examine Clusters

##### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, a name is assigned to each cluster

In [185]:
CT_merged.loc[CT_merged['Cluster Labels'] == 0, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Tennis Court,Playground,Yoga Studio,Dessert Shop,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden,Fried Chicken Joint


In [186]:
print ('Cluster 0 can be named as a spot for **SPORTS & ENTERTAINMENT**')

Cluster 0 can be named as a spot for **SPORTS & ENTERTAINMENT**


In [187]:
CT_merged.loc[CT_merged['Cluster Labels'] == 1, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,1,Ice Cream Shop,Garden,Hotel,Gym / Fitness Center,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Fried Chicken Joint,Food & Drink Shop


In [188]:
print ('Cluster 1 can be named as a spot for **EXERCISING & GOOD HEALTH**')

Cluster 1 can be named as a spot for **EXERCISING & GOOD HEALTH**


In [189]:
CT_merged.loc[CT_merged['Cluster Labels'] == 2, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,2,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden
1,Central Toronto,2,Hotel,Food & Drink Shop,Sandwich Place,Park,Clothing Store,Playground,Breakfast Spot,Grocery Store,Gym,Gourmet Shop
2,Central Toronto,2,Coffee Shop,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Park,Diner,Dessert Shop,Salon / Barbershop,Spa
3,Central Toronto,2,Pizza Place,Sandwich Place,Dessert Shop,Restaurant,Thai Restaurant,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Discount Store
5,Central Toronto,2,Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Liquor Store,Light Rail Station,Pizza Place,Supermarket,Sports Bar,Fried Chicken Joint
7,Central Toronto,2,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
8,Central Toronto,2,Coffee Shop,Sandwich Place,Café,Pizza Place,American Restaurant,Pharmacy,Liquor Store,Pub,Jewish Restaurant,Cosmetics Shop


In [190]:
print ('Cluster 2 can be named as a spot for **EATING & ENJOYING**')

Cluster 2 can be named as a spot for **EATING & ENJOYING**


### Analyzing all the top venues within all neighbourhoods in Boroughs in df_updated (Boroughs with the word 'Toronto')

In [191]:
CT_venues_all = getNearbyVenues (names=df_updated['Neighbourhood'], latitudes=df_updated['Latitude'], longitudes=df_updated['Longitude'],radius=500)
CT_venues_all

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
5,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,"The Danforth West,Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
7,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
8,"The Danforth West,Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
9,"The Danforth West,Riverdale",43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop


In [192]:
#Total No of venues fetched in all the Boroughs with the word 'Toronto' in the city of Toronto - size of CT_venues_all df
print ('Total No of venues fetched in all the Boroughs with the word *Toronto* in the city of Toronto is' , CT_venues_all.shape[0])

Total No of venues fetched in all the Boroughs with the word *Toronto* in the city of Toronto is 1700


In [124]:
#No of top venues returned for each neighbourhood in Central Toronto
CT_venues_all.groupby('Neighbourhood').count()


,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",46,46,46,46,46,46
Central Bay Street,88,88,88,88,88,88
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [193]:
#No of unique categories from all returned venues for all neighbourhoods in Central Toronto
print('There are {} unique venue categories'.format (len(CT_venues_all['Venue Category'].unique())))

There are 238 unique venue categories


In [194]:
#get 'Venue Category' as columns in a new df CT_hotspots_all
CT_hotspots_all = pd.get_dummies(CT_venues_all[['Venue Category']],prefix="",prefix_sep="")
CT_hotspots_all.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [195]:
#Add 'Neighbourhood' column to CT_hotspots_all df
CT_hotspots_all['Neighbourhood'] = CT_venues_all['Neighbourhood']

#move 'Neighbourhood' column to 1st column in CT_hotspots df
fixed_cols = [CT_hotspots_all.columns[-1]] + list(CT_hotspots_all.columns[:-1])
CT_hotspots_all = CT_hotspots_all[fixed_cols]

CT_hotspots_all.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [196]:
#Group rows in CT_hotspots_all df by 'Neighbourhood' + mean of the frequency of occurance of each venue category
CT_grp_all = CT_hotspots_all.groupby('Neighbourhood').mean().reset_index()
CT_grp_all

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.040000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.050000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.

In [197]:
#get only top 10 venues for each neighbourhood in all neighbourhoods in Boroughs that include in df_updated in to a new df

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
CT_venues_sorted_all = pd.DataFrame(columns=columns)
CT_venues_sorted_all['Neighbourhood'] = CT_grp_all['Neighbourhood']

for ind in np.arange(CT_grp_all.shape[0]):
    CT_venues_sorted_all.iloc[ind, 1:] = return_most_common_venues(CT_grp_all.iloc[ind, :], num_top_venues)

CT_venues_sorted_all

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,American Restaurant,Bakery,Restaurant,Hotel,Burger Joint,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Bakery,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Italian Restaurant,Farmers Market
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Yoga Studio,Intersection,Performing Arts Venue,Caribbean Restaurant,Stadium,Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Skate Park,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Terminal,Airport Lounge,Airport Service,Boutique,Coffee Shop,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Airport Gate
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Pet Store,Breakfast Spot
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Middle Eastern Restaurant,Sandwich Place,Burger Joint,Japanese Restaurant,Ice Cream Shop,Spa,Bar
7,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Bar,Vietnamese Restaurant,Chinese Restaurant,Cocktail Bar
8,Christie,Café,Grocery Store,Park,Convenience Store,Coffee Shop,Baby Store,Nightclub,Diner,Italian Restaurant,Restaurant
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Restaurant,Gay Bar,Men's Store,Gym,Pub,Mediterranean Restaurant,Bubble Tea Shop


#### Cluster all 38 Neighbourhoods 

In [198]:
# set number of clusters
kclusters = 3

CT_grp_clustering_all = CT_grp_all.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grp_clustering_all)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [201]:
# add clustering labels
CT_venues_sorted_all.insert(0, 'Cluster Labels', kmeans.labels_)

CT_merged_all = df_updated

# merge CT_venues_sorted with CT_data to add latitude/longitude for each neighborhood
CT_merged_all = CT_merged_all.join(CT_venues_sorted_all.set_index('Neighbourhood'), on='Neighbourhood')

CT_merged_all.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store,Spa
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,1,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [200]:
# create map
add_CT = 'Central Toronto, Toronto'
geolocator = Nominatim (user_agent='Toronto_explorer')
loc = geolocator.geocode(add_CT)
lat = loc.latitude
long = loc.longitude
map_clusters_CT_2 = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged_all['Latitude'], CT_merged_all['Longitude'], CT_merged_all['Neighbourhood'], CT_merged_all['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_CT_2)
       
map_clusters_CT_2

KeyError: 'Cluster Labels'

### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, a name is assigned to each cluster

In [139]:
CT_merged_all.loc[CT_merged_all['Cluster Labels'] == 0, CT_merged_all.columns[[1] + list(range(5, CT_merged_all.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
10,Downtown Toronto,0,Park,Playground,Trail,Dog Run,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
23,Central Toronto,0,Jewelry Store,Trail,Park,Sushi Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Yoga Studio


In [143]:
print ('Cluster 0 can be named as a spot for **LEISURE & ENTERTAINMENT**')

Cluster 0 can be named as a spot for **LEISURE & ENTERTAINMENT**


In [140]:
CT_merged_all.loc[CT_merged_all['Cluster Labels'] == 1, CT_merged_all.columns[[1] + list(range(5, CT_merged_all.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Health Food Store,Pub,Trail,Neighborhood,Other Great Outdoors,Falafel Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1,East Toronto,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Fruit & Vegetable Store,Juice Bar,Liquor Store,Spa
2,East Toronto,1,Park,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint,Brewery
3,East Toronto,1,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Brewery,Seafood Restaurant
5,Central Toronto,1,Playground,Hotel,Clothing Store,Food & Drink Shop,Grocery Store,Park,Gym,Breakfast Spot,Sandwich Place,Falafel Restaurant
6,Central Toronto,1,Coffee Shop,Yoga Studio,Bagel Shop,Park,Clothing Store,Dessert Shop,Chinese Restaurant,Rental Car Location,Diner,Salon / Barbershop
7,Central Toronto,1,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Café,Thai Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Deli / Bodega
8,Central Toronto,1,Tennis Court,Playground,Convenience Store,Doner Restaurant,Fish Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,Central Toronto,1,Pub,Coffee Shop,Liquor Store,Light Rail Station,Sushi Restaurant,Supermarket,Sports Bar,Fried Chicken Joint,American Restaurant,Vietnamese Restaurant
11,Downtown Toronto,1,Coffee Shop,Restaurant,Park,Café,Italian Restaurant,Bakery,Pub,Pizza Place,Pet Store,Breakfast Spot


In [144]:
print ('Cluster 1 can be named as a spot for **EATING & ENJOYING**')

Cluster 1 can be named as a spot for **EATING & ENJOYING**


In [145]:
CT_merged_all.loc[CT_merged_all['Cluster Labels'] == 2, CT_merged_all.columns[[1] + list(range(5, CT_merged_all.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Ice Cream Shop,Garden,Yoga Studio,Doner Restaurant,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


In [147]:
print ('Cluster 2 can be named as a spot of **MISCELLANEOUS**')

Cluster 2 can be named as a spot of **MISCELLANEOUS**
